# Code to calculate tissue composition in GT tissue map, vs pix2pix, pyramid pix2pix, and I2SB:
### Since this is to calculate bulk tissue composition so that it is invariant to the slight pixel offsets vs GT, all you have to do is just sum up the masks and then represent the tissue composition as % by dividing by the total pixel area.

In [1]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
import cv2

In [6]:
# for i2sb, it is tmp, when finish sampling I2SB, do evaluation on all data:
gt_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\val\HE\classification_10172023"
p2p_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\pix2pix_sample1_only\classification_10162023"
# p_p2p_pth = # tbd
# i2sb_pth = # tbd

gt_images = [os.path.join(gt_pth,x) for x in os.listdir(gt_pth) if x.endswith(".png")]
p2p_images = [os.path.join(p2p_pth,x) for x in os.listdir(p2p_pth) if x.endswith(".png")]
# p_p2p_images = [os.path.join(p_p2p_pth,x) for x in os.listdir(p_p2p_pth) if x.endswith(".png")]
# i2sb_images = [os.path.join(i2sb_pth,x) for x in os.listdir(i2sb_pth) if x.endswith(".png")]

In [7]:
def return_tissue_composition(image_directory):
    pixel_value_counts = np.zeros(9, dtype=int)
    for filename in tqdm(image_directory,colour="red"):
        image = cv2.imread(filename, 0)
        unique, counts = np.unique(image, return_counts=True)
        pixel_value_counts[unique] += counts
    tissue_comp = [x/pixel_value_counts.sum() for x in pixel_value_counts]
    tissue_comp = [x*100 for x in tissue_comp]
    tissue_comp = tissue_comp[1:]
    return tissue_comp

In [8]:
gt_pixel_value_counts = return_tissue_composition(gt_images)
p2p_pixel_value_counts = return_tissue_composition(p2p_images)

100%|██████████| 9951/9951 [01:38<00:00, 101.00it/s]


In [12]:
def find_MAE(list1,list2):
    if len(list1) != len(list2):
        raise ValueError("Lists must be of equal length")
    absolute_differences = [abs(a - b) for a, b in zip(list1, list2)]
    mae = sum(absolute_differences) / len(absolute_differences)
    return mae

[1.145437676040306, 0.7418683104193096, 1.7018703019120685, 6.052104913153449, 56.5369057118701, 14.424007881875973, 19.155930092863716, 0.2418751118650774]
[1.288278923157327, 0.8155905548782594, 1.9711781161391082, 6.332270048822237, 56.27300992272674, 14.037663376982955, 18.98074962701828, 0.30125943027508606]
[1.0104320440261694, 0.5238963860024209, 1.7843640502242797, 6.012669100255445, 56.97376490405901, 14.52144073903369, 18.927915303270147, 0.2455174731288309]
[0.8774785082156473, 0.6823543088182877, 1.632509029351243, 5.783905099832931, 56.67458121087273, 14.869671234017387, 19.27848848885908, 0.20101212003269145]


In [19]:
find_MAE(gt_pixel_value_counts,p2p_pixel_value_counts)

### Note: 1~8 in order: islet duct vessels fat acini collagen whitespace nerves

In [25]:
p2p_pixel_value_counts

MAE is 0.1764743117848524


In [26]:
gt_pixel_value_counts

MAE is 0.29613308584575077
